In [ ]:

import torch
from sklearn import datasets
import pandas as pd


# Cargar el dataset Iris
iris = datasets.load_iris()

# Convertirlo a un DataFrame de pandas
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['species'] = iris.target  # Agregar la columna de etiquetas numéricas
df['species'] = df['species'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})  # Convertir a nombres de especies
N = df.shape[0]
# Mostrar las primeras filas
df.head()


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [ ]:

N

150

In [ ]:

# Extraer las columnas usando NumPy
import numpy as np
x = np.array(df['petal length (cm)'])  # Definir x como petal length
y = np.array(df['petal width (cm)'])   # Definir y como petal width

# Mostrar los primeros valores
print("x (Petal Length):", x[:5])
print("y (Petal Width):", y[:5])

# Mezcla los índices
idx = np.arange(N)
np.random.shuffle(idx)

# Usa los primeros 80% de índices para entrenamiento
train_idx = idx[:int(N * 0.8)]
# Usa el 20% restante para validación
val_idx = idx[int(N * 0.8):]

# Generar los conjuntos de entrenamiento y validación
x_train, y_train = x[train_idx], y[train_idx]
x_val, y_val = x[val_idx], y[val_idx]

# Mostrar tamaños de los conjuntos
print(f"Train set: {x_train.shape}, {y_train.shape}")
print(f"Validation set: {x_val.shape}, {y_val.shape}")

x (Petal Length): [1.4 1.4 1.3 1.5 1.4]
y (Petal Width): [0.2 0.2 0.2 0.2 0.2]
Train set: (120,), (120,)
Validation set: (30,), (30,)


# Fase de entrenamiento


In [ ]:

x_train_tensor = torch.as_tensor(x_train)
y_train_tensor = torch.as_tensor(y_train)


device = 'cpu'


In [ ]:
import torch

# Supongamos que x_train_tensor y y_train_tensor están definidos
# y tienen la misma cantidad de elementos

# Define la tasa de aprendizaje
lr = 0.001

# Número de épocas
n_epochs = 1000

# Inicialización de parámetros
b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
w = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)

# Bucle de entrenamiento
for epoch in range(n_epochs):
    for i in range(len(x_train_tensor)):  # Iteramos sobre cada punto de datos
        x_i = x_train_tensor[i]  # Seleccionamos un solo punto x
        y_i = y_train_tensor[i]  # Seleccionamos su correspondiente y

        # Paso 1: Predicción del modelo
        yhat = b + w * x_i

        # Paso 2: Cálculo de la pérdida (error cuadrático medio para un solo punto)
        loss = (yhat - y_i) ** 2

        # Paso 3: Cálculo de gradientes
        loss.backward()

        # Paso 4: Actualización de parámetros
        with torch.no_grad():
            b -= lr * b.grad
            w -= lr * w.grad

        # Limpiar gradientes antes de la siguiente iteración
        b.grad.zero_()
        w.grad.zero_()

    # Imprimir cada 50 épocas
    #if (epoch + 1) % 50 == 0:
    if (epoch + 1) in [1, 10, 30] or (epoch + 1) % 50 == 0:
        print(f"Época {epoch+1}: b = {b.item()}, w = {w.item()}, pérdida = {loss.item()}")

Época 1: b = -0.6071049571037292, w = 0.4596485495567322, pérdida = 0.06593582779169083
Época 10: b = -0.5423060059547424, w = 0.45591437816619873, pérdida = 0.03848109766840935
Época 30: b = -0.45921024680137634, w = 0.4379741847515106, pérdida = 0.01803557202219963
Época 50: b = -0.41937655210494995, w = 0.42937415838241577, pérdida = 0.0109491478651762
Época 100: b = -0.38853487372398376, w = 0.42271554470062256, pérdida = 0.006670767907053232
Época 150: b = -0.38362768292427063, w = 0.4216560423374176, pérdida = 0.00608730036765337
Época 200: b = -0.3828471302986145, w = 0.42148762941360474, pérdida = 0.00599692715331912
Época 250: b = -0.3827223479747772, w = 0.42146068811416626, pérdida = 0.005982557777315378
Época 300: b = -0.3827029764652252, w = 0.42145639657974243, pérdida = 0.005980340298265219
Época 350: b = -0.3827022910118103, w = 0.4214562773704529, pérdida = 0.005980262067168951
Época 400: b = -0.3827022910118103, w = 0.4214562773704529, pérdida = 0.005980262067168951
É

In [ ]:
from scipy import stats
# Realizar la regresión lineal
slope, intercept, r_value, p_value, std_err = stats.linregress(x_train, y_train)

# Mostrar los resultados
print(f"Coeficiente de pendiente (w): {slope}")
print(f"Coeficiente de intercepción (b): {intercept}")
print(f"Coeficiente de correlación (r): {r_value}")
print(f"Valor p: {p_value}")
print(f"Error estándar: {std_err}")


Coeficiente de pendiente (w): 0.41978267197769203
Coeficiente de intercepción (b): -0.3803812215275777
Coeficiente de correlación (r): 0.9581396637068823
Valor p: 6.144178910344734e-66
Error estándar: 0.011547231530290233


# Gradiente descendete (la palabra estocastico no va)

In [ ]:

lr = 0.01

# Step 0 - Initializes parameters "b" and "w" randomly
torch.manual_seed(42)
b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
w = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
(b, w)


(tensor([0.3367], requires_grad=True), tensor([0.1288], requires_grad=True))

In [ ]:
# Supongamos que x_train_tensor y y_train_tensor están definidos
# y tienen la misma cantidad de elementos

# Define la tasa de aprendizaje
lr = 0.001

# Número de épocas
n_epochs = 31000

# Inicialización de parámetros
b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
w = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)


for epoch in range(n_epochs):
    # Step 1 - Computes model's predicted output - forward pass
    yhat = b + w * x_train_tensor

    # Step 2 - Computes the loss
    # We are using ALL data points, so this is BATCH gradient
    # descent. How wrong is our model? That's the error!
    error = (yhat - y_train_tensor)
    # It is a regression, so it computes mean squared error (MSE)
    loss = (error ** 2).mean()

    # Step 3 - Computes gradients for both "b" and "w"
    loss.backward()

    # Step 4 - Updates parameters using gradients and
    # the learning rate.
    with torch.no_grad():
        b -= lr * b.grad # b = b -lr * b.grad
        w -= lr * w.grad

    # Clears gradients
    b.grad.zero_()
    w.grad.zero_()

    # Prints values every 50 iterations
    if (epoch + 1) in [1, 10, 30] or (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1}: b = {b.item()}, w = {w.item()}, loss = {loss.item()}")

Epoch 1: b = 0.23468923568725586, w = 0.23227083683013916, loss = 0.15806939911097675
Epoch 10: b = 0.23610414564609528, w = 0.24685508012771606, loss = 0.1326122135872609
Epoch 30: b = 0.2364180088043213, w = 0.2663455307483673, loss = 0.11165447485945713
Epoch 50: b = 0.23458245396614075, w = 0.27597612142562866, loss = 0.10641250439798669
Epoch 100: b = 0.22671200335025787, w = 0.2846895158290863, loss = 0.10332704513391583
Epoch 150: b = 0.21773993968963623, w = 0.28773370385169983, loss = 0.10152326742886333
Epoch 200: b = 0.2087172269821167, w = 0.28988566994667053, loss = 0.09980146427627391
Epoch 250: b = 0.1998026967048645, w = 0.29187580943107605, loss = 0.09813206166491187
Epoch 300: b = 0.1910187304019928, w = 0.29381605982780457, loss = 0.09651283763783525
Epoch 350: b = 0.18236707150936127, w = 0.29572388529777527, loss = 0.09494227571641446
Epoch 400: b = 0.17384639382362366, w = 0.2976023852825165, loss = 0.09341892358040899
Epoch 450: b = 0.1654546707868576, w = 0.2994

Batch

In [ ]:
# Define la tasa de aprendizaje
lr = 0.001

# Número de épocas
n_epochs = 20000

# Tamaño del mini-batch
batch_size = 32

# Inicialización de parámetros
b = torch.randn(1, requires_grad=True, dtype=torch.float32)
w = torch.randn(1, requires_grad=True, dtype=torch.float32)

# Bucle de entrenamiento
for epoch in range(n_epochs):
    # Mezclar los datos en cada época (opcional pero recomendado)
    indices = torch.randperm(len(x_train_tensor))
    x_train_tensor = x_train_tensor[indices]
    y_train_tensor = y_train_tensor[indices]

    # Iterar sobre mini-lotes
    for i in range(0, len(x_train_tensor), batch_size):
        # Seleccionar el mini-batch actual
        x_batch = x_train_tensor[i:i + batch_size]
        y_batch = y_train_tensor[i:i + batch_size]

        # Paso 1: Predicción del modelo
        yhat = b + w * x_batch

        # Paso 2: Cálculo de la pérdida (error cuadrático medio para el mini-batch)
        loss = ((yhat - y_batch) ** 2).mean()

        # Paso 3: Cálculo de gradientes
        loss.backward()

        # Paso 4: Actualización de parámetros
        with torch.no_grad():
            b -= lr * b.grad
            w -= lr * w.grad

        # Limpiar gradientes antes de la siguiente iteración
        b.grad.zero_()
        w.grad.zero_()

    # Imprimir cada 50 épocas o en épocas específicas
    if (epoch + 1) in [1, 10, 30] or (epoch + 1) % 50 == 0:
        print(f"Época {epoch+1}: b = {b.item()}, w = {w.item()}, pérdida = {loss.item()}")

Época 1: b = -1.099576711654663, w = -0.08388510346412659, pérdida = 8.526732789194492
Época 10: b = -0.9899784922599792, w = 0.38290342688560486, pérdida = 0.6549741116400277
Época 30: b = -0.9420388340950012, w = 0.534775972366333, pérdida = 0.06709950405212052
Época 50: b = -0.9267549514770508, w = 0.5401038527488708, pérdida = 0.07022358863318233
Época 100: b = -0.8943546414375305, w = 0.5332629680633545, pérdida = 0.06210325433472904
Época 150: b = -0.8638348579406738, w = 0.5264601707458496, pérdida = 0.06983461061741576
Época 200: b = -0.8352720141410828, w = 0.5201169848442078, pérdida = 0.04551444432427321
Época 250: b = -0.8083080649375916, w = 0.5140758752822876, pérdida = 0.09311152718669487
Época 300: b = -0.7828769683837891, w = 0.5087029337882996, pérdida = 0.09916774763571824
Época 350: b = -0.7591798305511475, w = 0.5033190846443176, pérdida = 0.07749742687256321
Época 400: b = -0.7366461753845215, w = 0.49828040599823, pérdida = 0.06282728057357174
Época 450: b = -0.7

# Simplificación


In [ ]:

# Extraer las columnas usando NumPy
import numpy as np
x = np.array(df['petal length (cm)'])  # Definir x como petal length
y = np.array(df['petal width (cm)'])   # Definir y como petal width

# Mostrar los primeros valores
print("x (Petal Length):", x[:5])
print("y (Petal Width):", y[:5])

# Mezcla los índices
idx = np.arange(N)
np.random.shuffle(idx)

# Usa los primeros 80% de índices para entrenamiento
train_idx = idx[:int(N * 0.8)]
# Usa el 20% restante para validación
val_idx = idx[int(N * 0.8):]

# Generar los conjuntos de entrenamiento y validación
x_train, y_train = x[train_idx], y[train_idx]
x_val, y_val = x[val_idx], y[val_idx]

# Mostrar tamaños de los conjuntos
print(f"Train set: {x_train.shape}, {y_train.shape}")
print(f"Validation set: {x_val.shape}, {y_val.shape}")

x_train_tensor = torch.as_tensor(x_train)
y_train_tensor = torch.as_tensor(y_train)

x (Petal Length): [1.4 1.4 1.3 1.5 1.4]
y (Petal Width): [0.2 0.2 0.2 0.2 0.2]
Train set: (120,), (120,)
Validation set: (30,), (30,)


In [ ]:
import torch
import torch.optim as optim

# Define el dispositivo (CPU o GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Define la tasa de aprendizaje (eta)
lr = 0.01

# 2. Inicializa los parámetros "b" y "w" aleatoriamente
torch.manual_seed(42)  # Para reproducibilidad
b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
w = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)

# 3. Define un optimizador SGD para actualizar los parámetros
optimizer = optim.SGD([b, w], lr=lr)

# 4. Define el número de épocas
n_epochs = 10000

# 5. Bucle de entrenamiento
for epoch in range(n_epochs):
    # Paso 1 - Calcula la salida predicha por el modelo (forward pass)
    yhat = b + w * x_train_tensor

    # Paso 2 - Calcula la pérdida
    # Estamos usando TODOS los puntos de datos, por lo que esto es
    # Gradiente Descendente por Lotes (Batch Gradient Descent).
    error = (yhat - y_train_tensor)
    # Es una regresión, por lo que calculamos el error cuadrático medio (MSE)
    loss = (error ** 2).mean()

    # Paso 3 - Calcula los gradientes para los parámetros "b" y "w"
    loss.backward()

    # Paso 4 - Actualiza los parámetros usando los gradientes y
    # la tasa de aprendizaje. ¡No más actualización manual!
    # Step 4 - Updates parameters using gradients and
    # the learning rate. No more manual update!
    # with torch.no_grad():
    # b -= lr * b.grad
    # w -= lr * w.grad
    optimizer.step()

    # Paso 5 - Limpia los gradientes para la siguiente iteración
    optimizer.zero_grad()
    # No more telling Pytorch to let gradients go!
    # b.grad.zero_()
    # w.grad.zero_()

# 6. Imprime los parámetros finales
print(f"Parámetro b: {b.item()}")
print(f"Parámetro w: {w.item()}")

Parámetro b: -0.3803756535053253
Parámetro w: 0.4197814166545868


#Definición de función de perdida



In [ ]:
import torch.nn as nn  # Importa el módulo nn para usar MSELoss
import torch
import torch.optim as optim

In [ ]:
loss_fn = nn.MSELoss(reduction='mean')
# This is a random example to illustrate the loss function
predictions = torch.tensor([0.5, 1.0])
labels = torch.tensor([2.0, 1.3])
loss_fn(predictions, labels)

tensor(1.1700)

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn


# Define el dispositivo (CPU o GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(42)  # Para reproducibilidad
b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
w = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)

# 2. Inicializa los parámetros "b" y "w" aleatoriamente

# Define el dispositivo (CPU o GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Define la tasa de aprendizaje (eta)
lr = 0.01

# 2. Inicializa los parámetros "b" y "w" aleatoriamente
torch.manual_seed(42)  # Para reproducibilidad
b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
w = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)

# 3. Define un optimizador y un error SGD para actualizar los parámetros
optimizer = optim.SGD([b, w], lr=lr)

# Defines an MSE loss function
loss_fn = nn.MSELoss(reduction='mean')

# 4. Define el número de épocas
n_epochs = 10000

# 5. Bucle de entrenamiento
for epoch in range(n_epochs):
    # Paso 1 - Calcula la salida predicha por el modelo (forward pass)
    yhat = b + w * x_train_tensor

    # Paso 2 - Calcula la pérdida
    # Estamos usando TODOS los puntos de datos, por lo que esto es
    # Gradiente Descendente por Lotes (Batch Gradient Descent).
    # error = (yhat - y_train_tensor)
    # Es una regresión, por lo que calculamos el error cuadrático medio (MSE)
    # loss = (error ** 2).mean()

    loss = loss_fn(yhat, y_train_tensor)

    # Paso 3 - Calcula los gradientes para los parámetros "b" y "w"
    loss.backward()

    # Paso 4 - Actualiza los parámetros usando los gradientes y
    # la tasa de aprendizaje. ¡No más actualización manual!
    # Step 4 - Updates parameters using gradients and
    # the learning rate. No more manual update!
    # with torch.no_grad():
    # b -= lr * b.grad
    # w -= lr * w.grad
    optimizer.step()

    # Paso 5 - Limpia los gradientes para la siguiente iteración
    optimizer.zero_grad()
    # No more telling Pytorch to let gradients go!
    # b.grad.zero_()
    # w.grad.zero_()

# 6. Imprime los parámetros finales
print(f"Parámetro b: {b.item()}")
print(f"Parámetro w: {w.item()}")


Parámetro b: -0.3803756535053253
Parámetro w: 0.4197814166545868


# Incluir el modelo


In [ ]:

class ManualLinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        # Define los parámetros "b" y "w" como nn.Parameter
        self.b = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))
        self.w = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))

    def forward(self, x):
        # Calcula la salida/predicción del modelo
        return self.b + self.w * x


In [ ]:
torch.manual_seed(42)
# Creates a "dummy" instance of our ManualLinearRegression model
dummy = ManualLinearRegression()
list(dummy.parameters()) # Método heredado de nn.Module

[Parameter containing:
 tensor([0.3367], requires_grad=True),
 Parameter containing:
 tensor([0.1288], requires_grad=True)]

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn


# Define el dispositivo (CPU o GPU)
# Define el dispositivo (CPU o GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Define la tasa de aprendizaje (eta)
lr = 0.01

# 2. Inicializa los parámetros "b" y "w" aleatoriamente
torch.manual_seed(42)  # Para reproducibilidad
# Ya no se requiere inicializar b y w, están dentro de la clase
#b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
#w = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
# Inicializo modelo
model = ManualLinearRegression()
print(model.state_dict())

# 3. Define un optimizador y un error SGD para actualizar los parámetros
optimizer = optim.SGD(model.parameters(), lr=lr)
#optimizer = optim.SGD([b, w], lr=lr)

# Defines an MSE loss function
loss_fn = nn.MSELoss(reduction='mean')

# 4. Define el número de épocas
n_epochs = 10000

# 5. Bucle de entrenamiento
for epoch in range(n_epochs):
    # Paso 1 - Calcula la salida predicha por el modelo (forward pass)
    # Ya no se corre lo siguiente:
    # yhat = b + w * x_train_tensor
    model.train() # set the model to training mode
    yhat = model(x_train_tensor)

    # Paso 2 - Calcula la pérdida
    # Estamos usando TODOS los puntos de datos, por lo que esto es
    # Gradiente Descendente por Lotes (Batch Gradient Descent).
    # error = (yhat - y_train_tensor)
    # Es una regresión, por lo que calculamos el error cuadrático medio (MSE)
    # loss = (error ** 2).mean()

    loss = loss_fn(yhat, y_train_tensor)

    # Paso 3 - Calcula los gradientes para los parámetros "b" y "w"
    loss.backward()

    # Paso 4 - Actualiza los parámetros usando los gradientes y
    # la tasa de aprendizaje. ¡No más actualización manual!
    # Step 4 - Updates parameters using gradients and
    # the learning rate. No more manual update!
    # with torch.no_grad():
    # b -= lr * b.grad
    # w -= lr * w.grad
    optimizer.step()

    # Paso 5 - Limpia los gradientes para la siguiente iteración
    optimizer.zero_grad()
    # No more telling Pytorch to let gradients go!
    # b.grad.zero_()
    # w.grad.zero_()

print(model.state_dict())

OrderedDict([('b', tensor([0.3367])), ('w', tensor([0.1288]))])
OrderedDict([('b', tensor([-0.3804])), ('w', tensor([0.4198]))])


# En la practica

In [ ]:

linear = nn.Linear(1, 1)
print(linear)

Linear(in_features=1, out_features=1, bias=True)


In [ ]:
class MyLinearRegression(nn.Module):
  def __init__(self):
    super().__init__()
    # Instead of our custom parameters, we use a Linear model
    # with a single input and a single output
    self.linear = nn.Linear(1, 1)
    def forward(self, x):
    # Now it only makes
      self.linear(x)

In [ ]:
torch.manual_seed(42)
dummy = MyLinearRegression()
list(dummy.parameters())


[Parameter containing:
 tensor([[0.7645]], requires_grad=True),
 Parameter containing:
 tensor([0.8300], requires_grad=True)]

In [ ]:
x_train_tensor = torch.as_tensor(x_train, dtype=torch.float32, device=device).reshape(-1, 1)
y_train_tensor = torch.as_tensor(y_train, dtype=torch.float32, device=device).reshape(-1, 1)

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn


# Define el dispositivo (CPU o GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Define la tasa de aprendizaje (eta)
lr = 0.01

# 2. Inicializa los parámetros "b" y "w" aleatoriamente
torch.manual_seed(42)  # Para reproducibilidad

# Inicializo modelo
model = nn.Sequential(nn.Linear(1, 1)).to(device)  # Mueve el modelo al dispositivo

# Imprime el estado inicial del modelo
print("Estado inicial del modelo:")
print(model.state_dict())

# 3. Define un optimizador y un error SGD para actualizar los parámetros
optimizer = optim.SGD(model.parameters(), lr=lr)

# Define una función de pérdida MSE
loss_fn = nn.MSELoss(reduction='mean')

# 4. Define el número de épocas
n_epochs = 10000

# 5. Bucle de entrenamiento
for epoch in range(n_epochs):
    # Paso 1 - Calcula la salida predicha por el modelo (forward pass)
    model.train()  # Establece el modelo en modo de entrenamiento
    yhat = model(x_train_tensor)

    # Paso 2 - Calcula la pérdida
    loss = loss_fn(yhat, y_train_tensor)

    # Paso 3 - Calcula los gradientes para los parámetros "b" y "w"
    loss.backward()

    # Paso 4 - Actualiza los parámetros usando los gradientes y la tasa de aprendizaje
    optimizer.step()

    # Paso 5 - Limpia los gradientes para la siguiente iteración
    optimizer.zero_grad()

# Imprime el estado final del modelo
print("\nEstado final del modelo:")
print(model.state_dict())


Estado inicial del modelo:
OrderedDict([('0.weight', tensor([[0.7645]])), ('0.bias', tensor([0.8300]))])

Estado final del modelo:
OrderedDict([('0.weight', tensor([[0.4179]])), ('0.bias', tensor([-0.3781]))])
